In [79]:
import sqlite3
import pandas as pd

In [124]:
conn = sqlite3.connect('congress.db')

In [169]:
def query2df(conn, statement):
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute(statement)
    r = c.fetchall()
    k = r[0].keys()
    c.close()
    return pd.DataFrame(r, columns=k)

In [170]:
query = '''
            SELECT *
                FROM all_congress
                WHERE chamber = 'house'
            ;
            '''

In [171]:
df = query2df(conn, query)

In [175]:
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'])

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6744 entries, 0 to 6743
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             6744 non-null   object        
 1   first_name     6744 non-null   object        
 2   middle_name    2606 non-null   object        
 3   last_name      6744 non-null   object        
 4   suffix         151 non-null    object        
 5   date_of_birth  6744 non-null   datetime64[ns]
 6   gender         6691 non-null   object        
 7   party          6744 non-null   object        
 8   state          6744 non-null   object        
 9   chamber        6744 non-null   object        
 10  congress       6744 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 579.7+ KB
